In [58]:
import pandas as pd
import MySQLdb
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import numpy as np
import jsonify

In [59]:
db = MySQLdb.connect("69.89.14.35","360feedback","360feedback@","ttn_central_v1" )
db.autocommit=False

In [60]:
df = pd.read_sql_query(sql="""SELECT 
    360_v1_respondent_feedback.*
FROM
    ttn_central_v1.360_v1_respondent_feedback
        JOIN
    360_v1_respondent_fs_rel_mapper ON 360_v1_respondent_fs_rel_mapper.RFRM_ID = 360_v1_respondent_feedback.RFRM_ID
WHERE
    360_v1_respondent_feedback.SSUBM_ID IS NOT NULL
        -- AND 360_v1_respondent_fs_rel_mapper.PROJ_ID = 525 
        and SSUBM_ID in (77,78)
        ;""",con=db)

In [61]:
df = df[~df.SUBJECTIVE.isin(["nopes","","."])]
df["sub_small"]=df.SUBJECTIVE.map(lambda x: x.lower())

In [62]:
df.head()

,RF_ID,CSM_ID,RATE,SSUBM_ID,SUBJECTIVE,RFRM_ID,sub_small
2,160992,None,0,77,Respect for individual - Treats team members w...,5942,respect for individual - treats team members w...
3,160993,None,0,78,Deliver the Promise- Focus on goals and commit...,5942,deliver the promise- focus on goals and commit...
4,161133,None,0,77,1. Humility: He is humble. He demonstrate this...,5940,1. humility: he is humble. he demonstrate this...
5,161134,None,0,78,1. Entrepreneurship: Though he demonstrate the...,5940,1. entrepreneurship: though he demonstrate the...
7,166236,None,0,78,"I feel he should demonstrate more of ""Team wor...",6509,"i feel he should demonstrate more of ""team wor..."


In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
df['tokens']=df.SUBJECTIVE.apply(lambda x: nltk.wordpunct_tokenize(x))
df.tokens.head()

2    [Respect, for, individual, -, Treats, team, me...
3    [Deliver, the, Promise, -, Focus, on, goals, a...
4    [1, ., Humility, :, He, is, humble, ., He, dem...
5    [1, ., Entrepreneurship, :, Though, he, demons...
7    [I, feel, he, should, demonstrate, more, of, "...
Name: tokens, dtype: object

In [64]:
stemmer = SnowballStemmer(language="english",ignore_stopwords=False)
wordnet_lemmatizer = WordNetLemmatizer()
df['stemmed'] = df.tokens.map(lambda x: " ".join([stemmer.stem(y) for y in x if y.isalpha()]))
df['stemmedarr'] = df.tokens.map(lambda x: set([stemmer.stem(y) for y in x if y.isalpha()]))
df.head()

,RF_ID,CSM_ID,RATE,SSUBM_ID,SUBJECTIVE,RFRM_ID,sub_small,tokens,stemmed,stemmedarr
2,160992,None,0,77,Respect for individual - Treats team members w...,5942,respect for individual - treats team members w...,"[Respect, for, individual, -, Treats, team, me...",respect for individu treat team member with di...,"{and, give, process, individu, share, an, in, ..."
3,160993,None,0,78,Deliver the Promise- Focus on goals and commit...,5942,deliver the promise- focus on goals and commit...,"[Deliver, the, Promise, -, Focus, on, goals, a...",deliv the promis focus on goal and commit entr...,"{and, on, goal, for, challeng, focus, proactiv..."
4,161133,None,0,77,1. Humility: He is humble. He demonstrate this...,5940,1. humility: he is humble. he demonstrate this...,"[1, ., Humility, :, He, is, humble, ., He, dem...",humil he is humbl he demonstr this valu over a...,"{and, all, over, humil, idea, him, pressur, im..."
5,161134,None,0,78,1. Entrepreneurship: Though he demonstrate the...,5940,1. entrepreneurship: though he demonstrate the...,"[1, ., Entrepreneurship, :, Though, he, demons...",entrepreneurship though he demonstr the hr ama...,"{and, control, process, had, challeng, rest, a..."
7,166236,None,0,78,"I feel he should demonstrate more of ""Team wor...",6509,"i feel he should demonstrate more of ""team wor...","[I, feel, he, should, demonstrate, more, of, ""...",i feel he should demonstr more of team work re...,"{and, this, own, help, give, feel, the, is, wi..."


In [65]:
excel = df[["tokens","stemmedarr"]]
excel.to_excel("/home/ranjan/Documents/test_gmr.xlsx")

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvec = TfidfVectorizer(min_df=.0025, max_df=.1, stop_words='english')
tvec_weights = tvec.fit_transform(df.stemmed.dropna())
weights = np.asarray(tvec_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': tvec.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(10)

ValueError: After pruning, no terms remain. Try a lower min_df or a higher max_df.

In [26]:
top_weight=weights_df.sort_values(by='weight', ascending=False).head(50)
top_weight.head(5)

NameError: name 'weights_df' is not defined

In [27]:
df["close"]=df.stemmedarr.map(lambda x: len([y for y in x if y in top_weight.term.values]))

NameError: global name 'top_weight' is not defined

In [28]:
df.sort_values(by="close",ascending=False).head()

KeyError: 'close'

In [29]:
stmts = pd.read_csv("/home/ranjan/git/python-practice/jupyter/searcg_stmt.csv")

In [30]:
stmts

,STATEMENT
0,Learning & Inner Excellence
1,Respect for Individual
2,Team work & Relationships
3,Deliver the Promise
4,Humility
5,Social Responsibility
6,Entrepreneurship


In [31]:
stmts['tokens']=stmts.STATEMENT.apply(lambda x: nltk.wordpunct_tokenize(x))
stmts['stemmedarr'] = stmts.tokens.map(lambda x: set([stemmer.stem(y) for y in x if y.isalpha()]))
stmts

,STATEMENT,tokens,stemmedarr
0,Learning & Inner Excellence,"[Learning, &, Inner, Excellence]","{excel, inner, learn}"
1,Respect for Individual,"[Respect, for, Individual]","{respect, individu, for}"
2,Team work & Relationships,"[Team, work, &, Relationships]","{work, relationship, team}"
3,Deliver the Promise,"[Deliver, the, Promise]","{the, promis, deliv}"
4,Humility,[Humility],{humil}
5,Social Responsibility,"[Social, Responsibility]","{respons, social}"
6,Entrepreneurship,[Entrepreneurship],{entrepreneurship}


In [32]:
stmts.loc[[2]].STATEMENT

2    Team work & Relationships
Name: STATEMENT, dtype: object

In [33]:
for i in range(stmts.shape[0]):
    print(i)
    cur_stmt = stmts.loc[[i]].stemmedarr.values[0]
    print(cur_stmt)
    df["stmt_{0}_close".format(i+1)]=df.stemmedarr.map(lambda x: len([y for y in x if y in cur_stmt]))
    df["stmt_{0}_close_arr".format(i+1)]=df.stemmedarr.map(lambda x: [y for y in x if y in cur_stmt])
    df["stmt_{0}_close_stat".format(i+1)] = df["stmt_{0}_close".format(i+1)].map(lambda x:x==len(cur_stmt))

0
set([u'excel', u'inner', u'learn'])
1
set([u'respect', u'individu', u'for'])
2
set([u'work', u'relationship', u'team'])
3
set([u'the', u'promis', u'deliv'])
4
set([u'humil'])
5
set([u'respons', u'social'])
6
set([u'entrepreneurship'])


In [34]:
df

,RF_ID,CSM_ID,RATE,SSUBM_ID,SUBJECTIVE,RFRM_ID,sub_small,tokens,stemmed,stemmedarr,...,stmt_4_close_stat,stmt_5_close,stmt_5_close_arr,stmt_5_close_stat,stmt_6_close,stmt_6_close_arr,stmt_6_close_stat,stmt_7_close,stmt_7_close_arr,stmt_7_close_stat
0,188889,None,0,77,Deliver the promise\r\nTeamwork and realtionship,6343,deliver the promise\r\nteamwork and realtionship,"[Deliver, the, promise, Teamwork, and, realtio...",deliv the promis teamwork and realtionship,"{and, realtionship, teamwork, deliv, promis, the}",...,True,0,[],False,0,[],False,0,[],False
1,167598,None,0,77,Learning & Deliver the promise: \r\nMr. Sharma...,6345,learning & deliver the promise: \r\nmr. sharma...,"[Learning, &, Deliver, the, promise, :, Mr, .,...",learn deliv the promis mr sharma is a fast lea...,"{a, learn, sharma, is, deliv, learner, ani, fa...",...,True,0,[],False,0,[],False,0,[],False
2,172862,None,0,77,1. Learning- Learn from all level irrespective...,6346,1. learning- learn from all level irrespective...,"[1, ., Learning, -, Learn, from, all, level, i...",learn learn from all level irrespect of hierar...,"{and, all, from, risk, execut, level, of, age,...",...,False,0,[],False,0,[],False,1,[entrepreneurship],True
3,168679,None,0,77,He has very high learning grasp and seen in va...,6347,he has very high learning grasp and seen in va...,"[He, has, very, high, learning, grasp, and, se...",he has veri high learn grasp and seen in vario...,"{and, high, in, seen, shown, teamwork, way, va...",...,False,0,[],False,0,[],False,0,[],False
4,202921,None,0,77,Teamwork and Relationships - Excellent rapport...,6349,teamwork and relationships - excellent rapport...,"[Teamwork, and, Relationships, -, Excellent, r...",teamwork and relationship excel rapport with u...,"{and, all, love, individu, attitud, respect, r...",...,False,0,[],False,0,[],False,0,[],False


In [39]:
df.to_excel("/home/ranjan/Documents/gmr_excel.xlsx",sheet_name="report")

In [35]:
stats = list()
for i in range(stmts.shape[0]):
    num = i+1
    obj = {"stmt":num,
           "statement":stmts.loc[[i]].STATEMENT.values[0],
           "status":df.groupby('stmt_{0}_close_stat'.format(num))['stmt_{0}_close_stat'.format(num)].count().to_dict()}
    stats.append(obj)
print(stats)

[{'status': {False: 5}, 'statement': 'Learning & Inner Excellence', 'stmt': 1}, {'status': {False: 4, True: 1}, 'statement': 'Respect for Individual', 'stmt': 2}, {'status': {False: 5}, 'statement': 'Team work & Relationships', 'stmt': 3}, {'status': {False: 3, True: 2}, 'statement': 'Deliver the Promise', 'stmt': 4}, {'status': {False: 5}, 'statement': 'Humility', 'stmt': 5}, {'status': {False: 5}, 'statement': 'Social Responsibility', 'stmt': 6}, {'status': {False: 4, True: 1}, 'statement': 'Entrepreneurship', 'stmt': 7}]


In [36]:
stat_df = pd.DataFrame.from_dict(stats)
stat_df

,statement,status,stmt
0,Learning & Inner Excellence,{False: 5},1
1,Respect for Individual,"{False: 4, True: 1}",2
2,Team work & Relationships,{False: 5},3
3,Deliver the Promise,"{False: 3, True: 2}",4
4,Humility,{False: 5},5
5,Social Responsibility,{False: 5},6
6,Entrepreneurship,"{False: 4, True: 1}",7


In [37]:
stat_df["status_F"]=stat_df.status.map(lambda x: x[False])
stat_df["status_T"]=stat_df.status.map(lambda x: x[True] if x.has_key(True) else 0)

In [38]:
stat_df

,statement,status,stmt,status_F,status_T
0,Learning & Inner Excellence,{False: 5},1,5,0
1,Respect for Individual,"{False: 4, True: 1}",2,4,1
2,Team work & Relationships,{False: 5},3,5,0
3,Deliver the Promise,"{False: 3, True: 2}",4,3,2
4,Humility,{False: 5},5,5,0
5,Social Responsibility,{False: 5},6,5,0
6,Entrepreneurship,"{False: 4, True: 1}",7,4,1
